# Import Libs

In [9]:
# -*- coding: utf-8 -*-
import pandas as pd, numpy as np
import pickle as pkl
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import hstack
from classCustom import ItemSelector,Scalers
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
import unicodedata
from sklearn.preprocessing import MultiLabelBinarizer

# Load CSV

In [2]:
# Recipe inputs
base_apprentissage_course_readyML_df = pd.read_csv('Data/base_apprentissage_course_readyML.csv')
base_apprentissage_course_readyML_df.head()

,nomProduit,descriptionProduit,prix,nomMarchand,PuericultureEnfants,EquipementsMaisonHighTech,SportsLoisirsCulture,BricolageDecorationJardinerieAnimalerie,ModeAccessoires,AutoMoto,...,Boulangerie,CharcuterieTraiteur,Cremerie,EpicerieSalee,EpicerieSucree,FruitsFrais,LegumesFrais,ProduitsMer,ProduitsDietetiques,Surgeles
0,Support Voiture Universel Kenu Airframe pour S...,NaN,24.99,Fnac,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Voiture U-Command Flash Flash Mc Queen Cars,Age minimum 3 ans,87.85,Fnac,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bateau avec seau, pelle et râteau",Le lot,8.99,MONOP,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bateau Ocean Dream,L'unité,6.99,MONOP,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Boisson gazeuse aux extraits végétaux,La bouteille de 1l,1.39,MONOP,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Load Dictionnaryies

In [3]:
# Liste des categories
list_cat={'PuericultureEnfants':1001,'EquipementsMaisonHighTech':1002,'SportsLoisirsCulture':1003,'BricolageDecorationJardinerieAnimalerie':1004,
          'ModeAccessoires':1005,'AutoMoto':1006,'BeauteSanteBienEtre':1007,'HotelRestaurantCafes':1008,'BanquesAssurances':1009,'VoyagesTransports':1010,
          'Services':1011,'Bebe':2000,'Boissons':2001,'Boucherie':2002,'Boulangerie':2003,'CharcuterieTraiteur':2004,'Cremerie':2005,'EpicerieSalee':2006,
          'EpicerieSucree':2007,'FruitsFrais':2008,'LegumesFrais':2009,'ProduitsMer':2010,'ProduitsDietetiques':2011,'Surgeles':2012
         }

list_col=['PuericultureEnfants','EquipementsMaisonHighTech',
          'SportsLoisirsCulture','BricolageDecorationJardinerieAnimalerie',
          'ModeAccessoires','AutoMoto','BeauteSanteBienEtre','HotelRestaurantCafes',
          'BanquesAssurances','VoyagesTransports','Services','Bebe','Boissons',
          'Boucherie','Boulangerie','CharcuterieTraiteur','Cremerie',
          'EpicerieSalee','EpicerieSucree','FruitsFrais','LegumesFrais','ProduitsMer','ProduitsDietetiques','Surgeles'
         ]
list_stop_word_french=['alors','au','aucuns','aussi','autre','avant','avec','avoir','bon','car','ce','cela','ces','ceux','chaque',
                       'ci','comme','comment','dans','des','du','dedans','dehors','depuis','devrait','doit','donc','dos','début',
                       'elle','elles','en','encore','essai','est','et','eu','fait','faites','fois','font','hors','ici','il','ils',
                       'je','juste','la','le','les','leur','là','ma','maintenant','mais','mes','mine','moins','mon','mot','même',
                       'ni','nommés','notre','nous','ou','où','par','parce','pas','peut','peu','plupart','pour','pourquoi','quand',
                       'que','quel','quelle','quelles','quels','qui','sa','sans','ses','seulement','si','sien','son','sont','sous',
                       'soyez','sujet','sur','ta','tandis','tellement','tels','tes','ton','tous','tout','trop','très','tu','voient',
                       'vont','votre','vous','vu','ça','étaient','état','étions','été','être']

# Data manipulation

In [4]:
def creationMultiCol(row):
    res=[]
    for col in list_col:
        if row[col]==1:
            res.append(list_cat[col])
    return res

def removeCols(df):
    result=df.loc[:,['nomProduit','descriptionProduit','prix','nomMarchand','multiClass']]
    return result

def normaliz(row):
    """ Normalize Text """
    # to lower case
    str1=row.lower()
    #print 'low_case : ', str1

    # Suppress number
    regex = re.compile('[^\D]')
    str1=regex.sub('', str1)
    #print 'only_letters : ', str1
    
    # Suppress punctuation
    regex = re.compile('[^a-z 0-9ÀÁÂÃÄÅàáâãäåÒÓÔÕÖØòóôõöøÈÉÊËèéêëÇçÌÍÎÏìíîïÙÚÛÜùúûüÿÑñ²°Ø×ßŠ]')
    try :
        str1=regex.sub('', str1)
    except UnicodeDecodeError:
        str1
    #print 'no_ponctuation : ', str1

    # Suppress stop words
    french_stopwords_ini=stopwords.words('french')
    french_stopwords_ini.extend(list_stop_word_french)
    french_stopwords = set(french_stopwords_ini)
    str1 = [token for token in str1.split(' ') if token.lower() not in french_stopwords]
    #print 'no_stop_words : ', str1

    # Suppress accent
    str1 = [unicodedata.normalize('NFD', unicode(word,'utf-8')).encode('ascii', 'ignore') for word in str1]    
    #print 'no_accent : ', str1

    # Stemming of words
    stemmer = FrenchStemmer()
    str1=[stemmer.stem(word) for word in str1]
    #print 'stemming : ', str1

    # Merging words
    str1=' '.join(str1)
    #print 'merge_list : ',str1
    
    return str1

#### Shaping data ####

def fromDFToInput(df):
    df.fillna('', inplace=True)
    
    df.loc[:,'nomProduit']=df.loc[:,'nomProduit'].apply(lambda x:normaliz(x))
    df.loc[:,'descriptionProduit']=df.loc[:,'descriptionProduit'].apply(lambda x:normaliz(x))
    
    df.loc[:,'NomDescr']=df.loc[:,'nomProduit']+' '+df.loc[:,'descriptionProduit']

    return df

# Pipeline creation

In [5]:
def pred(df):
    pipeline = Pipeline([
    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[

            # Pipeline for tf-idf the word data
            ('nom', Pipeline([
                ('selector', ItemSelector(key='nomProduit')),
                ('tfidf', TfidfVectorizer(min_df=.001,max_df=0.8,ngram_range=(1,2)))
            ])),
            # Pipeline for tf-idf the word data
            ('descr', Pipeline([
                ('selector', ItemSelector(key='descriptionProduit')),
                ('tfidf', TfidfVectorizer(min_df=.001,max_df=0.8,ngram_range=(1,1)))
            ])),
                    
            # Pipeline for tf-idf the word data
            ('descrNom', Pipeline([
                ('selector', ItemSelector(key='NomDescr')),
                ('tfidf', TfidfVectorizer(min_df=.001,max_df=0.8,ngram_range=(1,2)))
            ])),

            # Pipeline for log standard price
            ('prix', Pipeline([
                ('selector', ItemSelector(key='prix')),
                ('std', MinMaxScaler()),
                ('scale',Scalers())
            ]))    

        ],
                
    )),

    # Use a SVC classifier on the combined features
    ('rf', OneVsRestClassifier(MultinomialNB(),n_jobs=-1))
    ])
    mlb = MultiLabelBinarizer()
    pipeline.fit(df, mlb.fit_transform(df['multiClass'].values))
    return pipeline

# Saving Pickle

In [6]:
def save(model, name):
    pkl.dump(model, open("Models/multiNB.p", "wb" ) )
    print name+" saved"

In [7]:
df=base_apprentissage_course_readyML_df
df['multiClass']=df.apply(creationMultiCol, axis=1)
result=removeCols(df)
res = fromDFToInput(result)

In [8]:
# Modelling 
model=pred(res)
# Saving model
save(model, "multiNB")

/Library/Python/2.7/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Library/Python/2.7/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


multiNB saved
